<html>
<h1> Bergfex webscraping
</h1>
<body>We gather information about outdoor sport itineraries from <a href='https://www.bergfex.com/sommer/bern-region/touren/?isAjax=1&page=1'> Bergfex.com </a> in the region of Bern.<br />
    - The website displays 20 tours per page, with infos such as length, type of sport, rating, etc... <br />
    - Most informations are provided by users.<br />
<br />
We then clean the data, and do a bunch of simple analysis.
</body>
</html>

<html>
<h2> Part 1: Data scraping and cleaning
</h2>

</html>

In [4]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [5]:
# initializing the future colums of our dataframe with empty lists

title = []  # Title of the tour
difficulty = []  # difficulty (easy, medium, hard)
sport = []  # Sport type (hiking, sledging, snowshoe...)
length = []  # length in km
time = []  # tour time in hours:minutes
climb = []  # positive elevation climb in m
minmax = []  # minimum and maximum altitude of the tour in m
technique = []  # technique difficulty ratings (out of 6)
fitness = []  # fitness difficulty ratings (out of 6)
total_title = [] # will help extracting the ID of the tour

# all ratings are stored together so we will need this along the way:
rating = []  # list to store technique and fitness rating info

In [6]:
# looping over the 10 first pages of Bergfex
# each separate tour on the page is framed by a div tag with 'touren-details'

page_number = 20  # number of pages we want to scap through

for p in range(1, (page_number+1)):
    base_link = 'https://www.bergfex.com/sommer/bern-region/touren/?isAjax=1&page='
    link = base_link+str(p)  # going over p pages with numbers appended to the base link
    page = requests.get(link, timeout=8)
    print("scraped page", p)
    soup = BeautifulSoup(page.content, "html.parser")  # bs4.BeautifulSoup object
    tours = soup.findAll('div', {'class': 'touren-detail'})  # checks for the separate tours on the page

# For each page, we loop over each tour and fill our lists with info     
    for i in range(0,len(tours)):
        tour_1 = tours[i] # tour iterating over the 20 tours of the page

        tour_title = tour_1.findAll('a' ) # this gets the title out
        title.append([info.get_text().strip() for info in tour_title])
        total_title.append(tour_title) # this gets the full tourtitle information
        
        tour_diff = tour_1.findAll('span', {'class': 'tour-difficulty'}) #tour level difficulty
        difficulty.append([info.get_text().strip() for info in tour_diff])

        tour_type = tour_1.findAll('span', {'class': 'tour-type'}) # putting type into sports (should all be hiking)
        sport.append([info.get_text().strip() for info in tour_type])

        tour_stats = tour_1.findAll('div', {'class': 'tour-stats'}) # stats has 4 info binned together
        stat_text = [info.get_text().strip() for info in tour_stats]
        length.append(stat_text[0])
        time.append(stat_text[1])
        climb.append(stat_text[2])
        minmax.append(stat_text[3])
        
        tour_rating = tour_1.find_all("div", {'class': 'tour-rating'}) # getting the rating data 
        rating.append([info for info in tour_rating])   # it's a class name so we can't get_text.

scraped page 1
scraped page 2
scraped page 3
scraped page 4
scraped page 5
scraped page 6
scraped page 7
scraped page 8
scraped page 9
scraped page 10
scraped page 11
scraped page 12
scraped page 13
scraped page 14
scraped page 15
scraped page 16
scraped page 17
scraped page 18
scraped page 19
scraped page 20


In [7]:
print("number of tours collected:", len(title))

number of tours collected: 400


In [8]:
### Get the titles and IDs into a list and then a DF
links_title = []

for i in total_title:
    i = str(i)
    i = i.strip('[]')
    print(i)
    links_title.append(i)

id_df3 = pd.DataFrame(links_title)


<a class="h2" href="/sommer/bern-region/touren/rennrad/172138,heftig-schoenes-saxeten/" title="Heftig schönes Saxeten">Heftig schönes Saxeten</a>
<a class="h2" href="/sommer/bern-region/touren/wanderung/69998,vis-a-vis-der-engelhoerner/" title="Vis-à-vis der Engelhörner">Vis-à-vis der Engelhörner</a>
<a class="h2" href="/sommer/bern-region/touren/mountainbike/133092,bikeroute-nr-11-adelboden--tschentenalp/" title="Bikeroute Nr. 11 Adelboden - Tschentenalp">Bikeroute Nr. 11 Adelboden - Tschentenalp</a>
<a class="h2" href="/sommer/bern-region/touren/rodeln/131645,wispile-run/" title="Wispile Run">Wispile Run</a>
<a class="h2" href="/sommer/bern-region/touren/klettersteig/76710,allmenalp-klettersteig--kandersteg/" title="Allmenalp Klettersteig - Kandersteg">Allmenalp Klettersteig - Kandersteg</a>
<a class="h2" href="/sommer/bern-region/touren/wanderung/49281,wo-ehemals-berndeutsch-gesprochen-wurde/" title="Wo ehemals Berndeutsch gesprochen wurde">Wo ehemals Berndeutsch gesprochen wurde</a

In [9]:
### Strip out the ID --> in column ix 5 as 'str'
id_df2 = id_df3[0].str.split(' ',expand = True)
id_df1 = id_df2[2].str.split(',',expand = True)
id_df = id_df1[0].str.split('/',expand = True)
id_df[5][0]

## ---> id_df is holding the IDs

'172138'

In [10]:
# Dealing with the ratings for fitness and technique
# we need to extract Rating data which is "embedded" as a class name in the bs4.BeautifulSoup
rating[0]

[<div class="tour-rating">
 <div class="tour-rating-label">Technique</div>
 <div class="rating-circles rating-max6"><div class="rating-4"></div></div>
 </div>,
 <div class="tour-rating">
 <div class="tour-rating-label">Fitness</div>
 <div class="rating-circles rating-max6"><div class="rating-5"></div></div>
 </div>]

In [11]:
# converting into string
tour_rating_str = str(rating)

# taking out unnecessary info - each second rating is Technique or Fitness, starting with technique
rating_all_short = tour_rating_str.replace('<div class="tour-rating">\n<div class="tour-rating-label">Technique</div>\n<div class="rating-circles rating-max6"><div class="','').replace('<div class="tour-rating">\n<div class="tour-rating-label">Fitness</div>\n<div class="rating-circles rating-max6"><div class="','').replace('"></div></div>\n</div>',"")
rating_even_shorter = rating_all_short.replace('[','').replace(']','')


## splitting into lists
rating_list = rating_even_shorter.split(", ")

In [12]:
# filling the lists technique = [] and fitness = []  we defined in the beginning

for i in range(0,(len(rating_list))):
    if i == 0:
        technique.append(rating_list[i]) # first item goes into technique
    elif i % 2 == 0:
        technique.append(rating_list[i]) # then every second item as well
    else:
        fitness.append(rating_list[i]) # the other items go into fitness

In [13]:
# Forming a dataframe from our lists as columns:

hikes_df = pd.DataFrame(
    {'title': title,
     'difficulty': difficulty,
     'sport': sport,
     'length': length,
     'time': time,
     'climb': climb,
     'minmax': minmax,
     'technique': technique,
     'fitness': fitness
    })

In [14]:
#  First draft dataframe: brackets, units, minmax to separate
hikes_df

,title,difficulty,sport,length,time,climb,minmax,technique,fitness
0,[Heftig schönes Saxeten],[difficult],[Racing],26.64km,02:10h,890hm,"566 - 1,457m",rating-4,rating-5
1,[Vis-à-vis der Engelhörner],[difficult],[Hiking],16.79km,05:40h,670hm,"589 - 1,874m",rating-4,rating-5
2,[Bikeroute Nr. 11 Adelboden - Tschentenalp],[medium],[Mountainbike],16.89km,01:30h,829hm,"1,263 - 1,940m",rating-4,rating-3
3,[Wispile Run],[medium],[Sledge],8.16km,02:00h,9hm,"1,188 - 1,917m",rating-4,rating-4
4,[Allmenalp Klettersteig - Kandersteg],[difficult],[Via ferrata],1.42km,03:30h,533hm,"1,187 - 1,720m",rating-,rating-3
...,...,...,...,...,...,...,...,...,...
395,[In die Heimat von Karl Grunder],[easy],[Hiking],10.02km,03:10h,470hm,603 - 972m,rating-2,rating-2
396,[Das Emmental von der Moosegg aus gesehen],[medium],[Hiking],14.08km,04:20h,700hm,"597 - 1,009m",rating-2,rating-4
397,[Zum Ursprung der Emme],[medium],[Hiking],14.26km,04:50h,800hm,"984 - 1,715m",rating-3,rating-3
398,[Chasseral - The Queen of the Jura],[medium],[Mountainbike],37.88km,05:20h,"1,167hm","436 - 1,602m",rating-4,rating-4


<html>
<h2> Part 2: Data cleaning
</h2>
<body></body>
</html>

In [15]:
# Converting the text columns to strings for easier handling later

hikes_df[['title', 'difficulty', 'sport']] = hikes_df[['title', 'difficulty', 'sport']].astype('str')

In [16]:
# Removing the brackets

hikes_df['title'] = hikes_df['title'].str.strip('[]')
hikes_df['difficulty'] = hikes_df['difficulty'].str.strip('[]')
hikes_df['sport'] = hikes_df['sport'].str.strip('[]')

In [17]:
# For some reason '' appear now, removing those as well

hikes_df['title'] = pd.Series(hikes_df['title']).str.replace("'", '')
hikes_df['difficulty'] = pd.Series(hikes_df['difficulty']).str.replace("'", '')
hikes_df['sport'] = pd.Series(hikes_df['sport']).str.replace("'", '')

In [18]:
# The "string-cleaned" dataframe
hikes_df

,title,difficulty,sport,length,time,climb,minmax,technique,fitness
0,Heftig schönes Saxeten,difficult,Racing,26.64km,02:10h,890hm,"566 - 1,457m",rating-4,rating-5
1,Vis-à-vis der Engelhörner,difficult,Hiking,16.79km,05:40h,670hm,"589 - 1,874m",rating-4,rating-5
2,Bikeroute Nr. 11 Adelboden - Tschentenalp,medium,Mountainbike,16.89km,01:30h,829hm,"1,263 - 1,940m",rating-4,rating-3
3,Wispile Run,medium,Sledge,8.16km,02:00h,9hm,"1,188 - 1,917m",rating-4,rating-4
4,Allmenalp Klettersteig - Kandersteg,difficult,Via ferrata,1.42km,03:30h,533hm,"1,187 - 1,720m",rating-,rating-3
...,...,...,...,...,...,...,...,...,...
395,In die Heimat von Karl Grunder,easy,Hiking,10.02km,03:10h,470hm,603 - 972m,rating-2,rating-2
396,Das Emmental von der Moosegg aus gesehen,medium,Hiking,14.08km,04:20h,700hm,"597 - 1,009m",rating-2,rating-4
397,Zum Ursprung der Emme,medium,Hiking,14.26km,04:50h,800hm,"984 - 1,715m",rating-3,rating-3
398,Chasseral - The Queen of the Jura,medium,Mountainbike,37.88km,05:20h,"1,167hm","436 - 1,602m",rating-4,rating-4


In [19]:
# getting rid of the units (they're always the same anyway)

hikes_df['length'] = pd.Series(hikes_df['length']).str.replace("km", '')
hikes_df['time'] = pd.Series(hikes_df['time']).str.replace("h", '')
hikes_df['climb'] = pd.Series(hikes_df['climb']).str.replace("hm", '')
hikes_df['minmax'] = pd.Series(hikes_df['minmax']).str.replace("m", '')

In [20]:
# making 2 columns out of the last one, and removing the old minmax column

hikes_df[['min','max']] = hikes_df['minmax'].str.split("-",expand=True)
hikes_df = hikes_df.drop(columns=['minmax'])

# moving the ratings back to the end
hikes_df = hikes_df[['title', 'difficulty', 'sport', 'length', 'time', 'climb', 'min', 'max', 'technique', 'fitness']]


In [21]:
# The clean dataframe in string format

hikes_df


,title,difficulty,sport,length,time,climb,min,max,technique,fitness
0,Heftig schönes Saxeten,difficult,Racing,26.64,02:10,890,566,"1,457",rating-4,rating-5
1,Vis-à-vis der Engelhörner,difficult,Hiking,16.79,05:40,670,589,"1,874",rating-4,rating-5
2,Bikeroute Nr. 11 Adelboden - Tschentenalp,medium,Mountainbike,16.89,01:30,829,"1,263","1,940",rating-4,rating-3
3,Wispile Run,medium,Sledge,8.16,02:00,9,"1,188","1,917",rating-4,rating-4
4,Allmenalp Klettersteig - Kandersteg,difficult,Via ferrata,1.42,03:30,533,"1,187","1,720",rating-,rating-3
...,...,...,...,...,...,...,...,...,...,...
395,In die Heimat von Karl Grunder,easy,Hiking,10.02,03:10,470,603,972,rating-2,rating-2
396,Das Emmental von der Moosegg aus gesehen,medium,Hiking,14.08,04:20,700,597,"1,009",rating-2,rating-4
397,Zum Ursprung der Emme,medium,Hiking,14.26,04:50,800,984,"1,715",rating-3,rating-3
398,Chasseral - The Queen of the Jura,medium,Mountainbike,37.88,05:20,"1,167",436,"1,602",rating-4,rating-4


In [22]:
# converting to number data types:

# length
hikes_df['length'] = (hikes_df['length']).astype(float)

# Climb
# to handle the conversions we replace missing values by 0. Maybe nan would work too, but didn't find out how yet
hikes_df['climb'] = pd.Series(hikes_df['climb']).str.replace(",", '') # replace thousand separator
hikes_df['climb'] = pd.Series(hikes_df['climb']).str.replace("-", '0') # replaces - by 0
hikes_df['climb'] = (hikes_df['climb']).astype(int)


In [23]:
#min and max
hikes_df['min'] = pd.Series(hikes_df['min']).str.replace(",", '') # replace thousand separator
hikes_df['min'] = (hikes_df['min']).astype(int)
hikes_df['max'] = pd.Series(hikes_df['max']).str.replace(",", '') # replace thousand separator
hikes_df['max'] = (hikes_df['max']).astype(int)

In [24]:
# dropping activities that are longer than 24 hrs
#### Watchout - only run once!!??

filt = hikes_df[(hikes_df['time'] > '24:00')]
filt

index_todrop = hikes_df.index[hikes_df['time'] > '24:00'].tolist()

hikes_df = hikes_df.drop(index_todrop)
#len(hikes_df)
hikes_df

,title,difficulty,sport,length,time,climb,min,max,technique,fitness
0,Heftig schönes Saxeten,difficult,Racing,26.64,02:10,890,566,1457,rating-4,rating-5
1,Vis-à-vis der Engelhörner,difficult,Hiking,16.79,05:40,670,589,1874,rating-4,rating-5
2,Bikeroute Nr. 11 Adelboden - Tschentenalp,medium,Mountainbike,16.89,01:30,829,1263,1940,rating-4,rating-3
3,Wispile Run,medium,Sledge,8.16,02:00,9,1188,1917,rating-4,rating-4
4,Allmenalp Klettersteig - Kandersteg,difficult,Via ferrata,1.42,03:30,533,1187,1720,rating-,rating-3
...,...,...,...,...,...,...,...,...,...,...
395,In die Heimat von Karl Grunder,easy,Hiking,10.02,03:10,470,603,972,rating-2,rating-2
396,Das Emmental von der Moosegg aus gesehen,medium,Hiking,14.08,04:20,700,597,1009,rating-2,rating-4
397,Zum Ursprung der Emme,medium,Hiking,14.26,04:50,800,984,1715,rating-3,rating-3
398,Chasseral - The Queen of the Jura,medium,Mountainbike,37.88,05:20,1167,436,1602,rating-4,rating-4


In [25]:
# converting times to time objects

# pandas datetime doesn't handle times over 24 hours so we have to give him an alternative
try:
    hikes_df['time'] = pd.to_datetime(hikes_df['time'], format='%H:%M').dt.time
except ValueError:
    pass
# I want to get rid of seconds but it's bed time


In [26]:
type(hikes_df['technique'][0])

str

In [27]:
## rating technique
hikes_df['technique'] = pd.Series(hikes_df['technique']).str.replace("rating-", '')
hikes_df['technique'] = pd.to_numeric(hikes_df['technique'], errors='coerce')
hikes_df['technique'] = pd.Series(hikes_df['technique']).replace(np.nan, 0, regex = True)
hikes_df['technique'] = pd.Series(hikes_df['technique']).astype('int')

# rating fitness
hikes_df['fitness'] = pd.Series(hikes_df['fitness']).str.replace("rating-", '')
hikes_df['fitness'] = pd.to_numeric(hikes_df['fitness'], errors='coerce')
hikes_df['fitness'] = pd.Series(hikes_df['fitness']).replace(np.nan, 0, regex = True)
hikes_df['fitness'] = hikes_df['fitness'].astype('int')

In [28]:
# Finally the clean dataframe 

hikes_df

,title,difficulty,sport,length,time,climb,min,max,technique,fitness
0,Heftig schönes Saxeten,difficult,Racing,26.64,02:10:00,890,566,1457,4,5
1,Vis-à-vis der Engelhörner,difficult,Hiking,16.79,05:40:00,670,589,1874,4,5
2,Bikeroute Nr. 11 Adelboden - Tschentenalp,medium,Mountainbike,16.89,01:30:00,829,1263,1940,4,3
3,Wispile Run,medium,Sledge,8.16,02:00:00,9,1188,1917,4,4
4,Allmenalp Klettersteig - Kandersteg,difficult,Via ferrata,1.42,03:30:00,533,1187,1720,0,3
...,...,...,...,...,...,...,...,...,...,...
395,In die Heimat von Karl Grunder,easy,Hiking,10.02,03:10:00,470,603,972,2,2
396,Das Emmental von der Moosegg aus gesehen,medium,Hiking,14.08,04:20:00,700,597,1009,2,4
397,Zum Ursprung der Emme,medium,Hiking,14.26,04:50:00,800,984,1715,3,3
398,Chasseral - The Queen of the Jura,medium,Mountainbike,37.88,05:20:00,1167,436,1602,4,4


In [29]:
# the dataframe info sum up yay we managed!
hikes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 399 entries, 0 to 399
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       399 non-null    object 
 1   difficulty  399 non-null    object 
 2   sport       399 non-null    object 
 3   length      399 non-null    float64
 4   time        399 non-null    object 
 5   climb       399 non-null    int64  
 6   min         399 non-null    int64  
 7   max         399 non-null    int64  
 8   technique   399 non-null    int64  
 9   fitness     399 non-null    int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 54.3+ KB


#### <html>
<h2> Part 3: Downloading GeoData for the hikes
</h2>
<body>to create the for loop for all the 400 hikes</body>
</html>

In [30]:
### Imports
import requests
import re
from pykml import parser
from lxml import etree
from lxml import objectify

#### <html> 
<h3> Download the KML file
    </h3>
<body>We do this by parsing the url, and then converting into the the kml link that is used on the website.

- https://www.bergfex.com/downloads/gps/?type=&id=271548&fileType=kml
- Get file name from the url
    </body>
</html>

In [31]:
kml_link = ""
url = "https://www.bergfex.com/downloads/gps/"
url3= "https://www.bergfex.com/downloads/gps/?type=&amp;id=271548&amp;fileType=kml"
if url3.find('/'):
    print(url3.rsplit('/', 1)[1])
    kml_link = (url3.rsplit('/', 1)[1])
    new_s = kml_link.replace('=&amp;', '=&')
    print(new_s)

conc_kml_link = (url + new_s)
print(conc_kml_link )

?type=&amp;id=271548&amp;fileType=kml
?type=&id=271548&amp;fileType=kml
https://www.bergfex.com/downloads/gps/?type=&id=271548&amp;fileType=kml


#### <html> 
<h3>Write the data to a klm file locally
    </h3>
<body> 
- TO DO:
- This will be the file that we need to write our data from, and then add it to a data frame
    </body>
    </html>

In [32]:
r = requests.get(conc_kml_link, allow_redirects=True)
write_link = ( new_s + ".kml")
write_link
open(write_link, 'wb').write(r.content)

14653

#### <html> 
<h3>Create loop to download all entries
    </h3>
    <body> DONE </body>

In [33]:
## get list of all IDs
final_id = id_df.iloc[:,5]
final_id

0      172138
1       69998
2      133092
3      131645
4       76710
        ...  
395    508338
396     51623
397     50397
398     60329
399     48766
Name: 5, Length: 400, dtype: object

In [34]:
test_loop = final_id.iloc[0:3]
test_loop

0    172138
1     69998
2    133092
Name: 5, dtype: object

# Loop through hikes and save gpx files to folder
This loop uses the hike id's to navigate to each of the hike's pages. It then downloads the kml file to the folder "gpx_files" and saves it as a filename "Unique hike id.gpx"

In [37]:
## For loop to create: i) all kml files (400 items) --> take 'final_id', ii) for testing --> take 'test_loop'
## Saves the files in same directory as the notebook

for sport_id in final_id: 
    url_id = f"https://www.bergfex.com/downloads/gps/?type=&id={sport_id};fileType=kml"
    print(url_id)
    r = requests.get(url_id, allow_redirects=True)
    write_link = (sport_id + ".gpx")
    # Replace kml files with your directory path
    with open('gpx_files/'+ write_link, 'wb') as f:
        f.write(r.content)


https://www.bergfex.com/downloads/gps/?type=&id=172138;fileType=kml
https://www.bergfex.com/downloads/gps/?type=&id=69998;fileType=kml
https://www.bergfex.com/downloads/gps/?type=&id=133092;fileType=kml
https://www.bergfex.com/downloads/gps/?type=&id=131645;fileType=kml
https://www.bergfex.com/downloads/gps/?type=&id=76710;fileType=kml
https://www.bergfex.com/downloads/gps/?type=&id=49281;fileType=kml
https://www.bergfex.com/downloads/gps/?type=&id=133939;fileType=kml
https://www.bergfex.com/downloads/gps/?type=&id=522401;fileType=kml
https://www.bergfex.com/downloads/gps/?type=&id=162912;fileType=kml
https://www.bergfex.com/downloads/gps/?type=&id=383168;fileType=kml
https://www.bergfex.com/downloads/gps/?type=&id=658326;fileType=kml
https://www.bergfex.com/downloads/gps/?type=&id=133081;fileType=kml
https://www.bergfex.com/downloads/gps/?type=&id=156751;fileType=kml
https://www.bergfex.com/downloads/gps/?type=&id=70002;fileType=kml
https://www.bergfex.com/downloads/gps/?type=&id=1084

## Correcting the url path x

https://www.bergfex.com/downloads/gps/?type=&amp;id=271548&amp;amp;fileType=kml    # Truth
Loop through each file and scrape out the gpx coords, then save it to a list

In [39]:
import glob

# This works!!!
hikingtrail_name_list = []
lat_list = []
lon_list = []
hiking_trail = []
path = "gpx_files/*.gpx"
for fname in glob.glob(path):
    soup = BeautifulSoup(open(fname,'r'), 'lxml')
    parser = etree.XMLParser(recover=True, remove_blank_text=True)
#    file_name="kml_files/131687.kml"
    root = etree.parse(fname, parser)
    names = root.xpath('//kml:name', namespaces={'kml': "http://www.topografix.com/GPX/1/1"})

    for name in names:
        trail_name = name.text
        lat = trkpt['lat']
        lon = trkpt['lat']
        # append to list 
        hiking_trail.append([name.text, trkpt['lat'], trkpt['lon']] )
#       hiking_trail.append(trkpt['lat'])
#       hiking_trail.append(trkpt['lon'])

print(hiking_trail)                                                                             # Have type=&amp;id=123

AssertionError: ElementTree not initialized, missing root

#### <html> 
<h3> Read out the first postion of each file and add to DF
    </h3>

<body> Still to do </body>

In [ ]:
kml_link = ""
url = "https://www.bergfex.com/downloads/gps/"
url3= "https://www.bergfex.com/downloads/gps/?type=&amp;id=271548&amp;fileType=kml"
if url3.find('/'):
    print(url3.rsplit('/', 1)[1])
    kml_link = (url3.rsplit('/', 1)[1])
    new_s = kml_link.replace('=&amp;', '=&')
    print(new_s)

conc_kml_link = (url + new_s)
print(conc_kml_link )

#### <html>
<h2> Part 4: Get snow data
</h2>
<body>DONE</body>
</html>

In [ ]:
### all the imports

import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
### initializing the future colums of our dataframe with empty lists

snowlevel = []  # height in cm
location = []  # town and elevation of town

## no looping over several pages necessary

## only scrape first page
link = 'http://www.meteocentrale.ch/de/wetter/hitlisten/schneehoehen.html'
page = requests.get(link, timeout=10)
print(page.status_code)
soup = BeautifulSoup(page.content, "html.parser")  # bs4.BeautifulSoup object
hitlist = soup.findAll('table', {'class': 'hitlist'})  #bs4.element.ResultSet

In [ ]:
## get location
location_item = hitlist[0].findAll('a')
location.append([info.get_text().strip() for info in location_item])
loc = location[0]
#len(loc)
#loc

## get snowlevel
snowlevel_item = hitlist[0].findAll('td', {'class': 'value'}) #
snowlevel.append([info.get_text().strip() for info in snowlevel_item])
#len(snowlevel)
snow = snowlevel[0]
#len(snow)
#snow


## combine into DF
heights_df = pd.DataFrame({'location': loc,'snowlevel': snow})
heights_df

In [ ]:
### data manipulation: 

##remove cm, convert to 'int'
heights_df['snowlevel_in_cm']=pd.Series(heights_df['snowlevel']).str.replace(" cm", '')
heights_df['snowlevel_in_cm']=pd.Series(heights_df['snowlevel_in_cm']).astype(int)
#type(heights_df['snowlevel_in_cm'][0])
#heights_df

## split location into 'village' and 'elevation of village' and merge with previous DF
split_loc = pd.Series(heights_df['location']).str.split(',',n=2,expand = True)
merged_df = pd.merge(heights_df, split_loc, left_index=True, right_index=True)
#merged_df

## drop unused columns, rename final columns, remove unit, sort columns
intermediate_df1 = merged_df.iloc[:,[2,3,4]]
intermediate_df1.columns = ['snowlevel_in_cm', 'location', 'height_in_m']
intermediate_df1['height_in_m']=pd.Series(intermediate_df1['height_in_m']).str.replace(" m", '')
intermediate_df2 = pd.DataFrame(intermediate_df1, columns = ['location', 'height_in_m', 'snowlevel_in_cm'])



In [ ]:
### final dataframe

intermediate_df2

In [ ]:
### Export into csv

#intermediate_df2.to_csv(r'/Users/sd/Documents/0-Coding/sarah-dutschke/04-Visualization/Challenge/snow_level.csv', index = False)

#### <html>
<h2> Part 5: Data analysis 
</h2>
<body>(just to check the dataframe actually works)</body>
</html>

In [ ]:
# Number of entries by types of sports
hikes_df['sport'].value_counts().plot(kind='bar')

In [ ]:
# Creating a df for "Number of entries by sport type" + average tour length

sporttype = hikes_df.groupby('sport') \
       .agg({'sport':'count', 'length':'mean'}) \
       .rename(columns={'sport':'count','length':'average_length'}) \
       .reset_index()
sporttype

In [ ]:
# Difficulty

hikes_df['difficulty'].unique()

In [ ]:
hikes_df['difficulty'].value_counts().plot(kind='bar')

In [ ]:
# mean length for difficulty:

hikes_df.groupby('difficulty').mean()

In [ ]:
#df.reset_index().plot.scatter(x = 'index', y = 'value')
tour_len = sorted(pd.Series(hikes_df["length"]))
tour_climb = sorted(pd.Series(hikes_df["climb"]))
plt.plot(tour_len, tour_climb)
plt.ylabel('climb (m)')
plt.xlabel("length (km)")
plt.title('All activities')

In [ ]:
## Creating a df for deeper analysis for Hiking
# Filter for hiking
hiking = hikes_df[hikes_df.sport == "Hiking"]

# calculating the count by Fitness level and the average length 
fitness_length = hiking.groupby('fitness') \
       .agg({'fitness':'count', 'length':'mean'}) \
       .rename(columns={'fitness':'count','length':'average_length'}) \
       .reset_index()

# calculating weighted and normalized length
fitness_length['weighted_ave_length'] = fitness_length["count"] * fitness_length["average_length"]
fitness_length["normalized_weighted_length"] = (fitness_length["weighted_ave_length"] - fitness_length["weighted_ave_length"].min()) \
                                                / (fitness_length["weighted_ave_length"].max() - fitness_length["weighted_ave_length"].min()) * 10
fitness_length


In [ ]:
## Plotting Length vs. Fitness rating
#--> Conclusion: the routes with no fitness rating seems quite long. Investigate further

plt.plot(fitness_length["fitness"], fitness_length["average_length"])
plt.ylabel('average_length')
plt.xlabel("rating_fitness")
plt.title('Hiking: Average Length by Fitness Level Rating')

plt.show()

In [ ]:
# quality checking data / or realizing we don't understand the columns?

hikes_df["residual"]= hikes_df["climb"] - (hikes_df["max"] - hikes_df["min"])
hikes_df["residual"].plot.density()

#### <html>
<h2> Part 6: Dashboard
</h2>
<body>(pretty much a lot to do stil :-))</body>
</html>

In [ ]:
### import Dashboard libraries

# bokeh
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, CDSView, HoverTool, Range1d,GroupFilter
from bokeh.io import output_notebook
from bokeh.palettes import Dark2_8, magma
from bokeh.models import Legend, LegendItem
import itertools

# Panel as well
import panel as pn

pn.extension()
output_notebook()

In [ ]:
## Prepare the x/y axis for elevation graph

## x_axis made out of time duration
x = []
for n in hikes_df['time']:
    x.append(n)
xs = [[0,n] for n in x]

## y_axis made out of min and max altitude
y1 = []
y2 = []
for min_, max_ in zip(hikes_df['min'], hikes_df['max']):
    y1.append(min_)
    y2.append(max_)
ys = [[min_, max_] for min_, max_ in zip(hikes_df['min'], hikes_df['max'])]

#ys
#len(ys)
#xs
len(xs)

In [ ]:
## Add to DataFrame

hikes_df['ys'] = ys
hikes_df['xs'] = xs
#type(hikes_df['time'][0])

In [ ]:
### Preparing the plots

## Create a ColumnDataSource
main_cds = ColumnDataSource(hikes_df)

## Create the views for each difficulty level
easy_view = CDSView(source = main_cds, filters=[GroupFilter(column_name='difficulty', group = 'easy')])
medium_view = CDSView(source = main_cds,filters=[GroupFilter(column_name='difficulty', group = 'medium')])
difficult_view = CDSView(source = main_cds,filters=[GroupFilter(column_name='difficulty',group = 'difficult')])

## Create and configure the figure
easy_fig = figure(x_range=[0,hikes_df['time'].max()], 
                  y_range=(hikes_df['min'].min()-100,hikes_df['max'].max()+100), 
                plot_width= 400, plot_height=300,
                 x_axis_label = "Length in hrs",
                 y_axis_label = 'Altitude: from start to highest point',
                 toolbar_location = None)

#line_color = magma(201)

## Fill the figures
easy_fig.multi_line('xs', 'ys', color = 'firebrick', alpha = 0.5, line_width=1, source = main_cds, view = easy_view)

## Remaining aestetics
#easy_fig.legend.location = 'below'

show(easy_fig)

In [ ]:
type(hikes_df['length'][0])